In [ ]:
import cv2 as cv
import numpy as np
import glob, os
import math
import pandas as pd
from scipy.signal import savgol_filter

def choose_component(labels, stats, centroids, prev_xy, min_area=200):
    # labels: HxW ints; stats: (n,5); centroids: (n,2)
    # row 0 is background
    n = stats.shape[0]
    candidates = [i for i in range(1, n) if stats[i, cv.CC_STAT_AREA] >= min_area]
    if not candidates:
        return None
    if prev_xy is None:
        # largest area
        return max(candidates, key=lambda i: stats[i, cv.CC_STAT_AREA])
    px, py = prev_xy
    def score(i):
        cx, cy = centroids[i]
        d2 = (cx - px)**2 + (cy - py)**2
        # prefer proximity, tie-break by area
        return (d2, -stats[i, cv.CC_STAT_AREA])
    return min(candidates, key=score)

def orientation_from_moments(binmask):
    M = cv.moments(binmask, binaryImage=True)
    mu20, mu02, mu11 = M['mu20'], M['mu02'], M['mu11']
    denom = (mu20 - mu02)
    if (mu20 + mu02) == 0:
        return None
    angle = 0.5 * math.atan2(2 * mu11, denom if denom != 0 else 1e-9)
    # radians → degrees, in screen coords
    return np.degrees(angle)

def track_from_masks(mask_glob_pattern, fps=30.0, homography=None, kernel_size=5, min_area=200):
    paths = sorted(glob.glob(mask_glob_pattern))
    if not paths:
        raise FileNotFoundError(f"No masks found for pattern: {mask_glob_pattern}")

    k = cv.getStructuringElement(cv.MORPH_ELLIPSE, (kernel_size, kernel_size))
    rows = []
    prev_xy = None

    for frame_idx, p in enumerate(paths):
        m = cv.imread(p, cv.IMREAD_UNCHANGED)
        if m is None:
            rows.append((frame_idx, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan))
            continue

        # If mask is colored/labelled, binarize: nonzero → 1
        if m.ndim == 3:
            m = cv.cvtColor(m, cv.COLOR_BGR2GRAY)
        binm = (m > 0).astype(np.uint8)

        # Morphology to clean noise
        binm = cv.morphologyEx(binm, cv.MORPH_OPEN, k)
        binm = cv.morphologyEx(binm, cv.MORPH_CLOSE, k)

        nlabels, labels, stats, centroids = cv.connectedComponentsWithStats(binm, connectivity=8)
        comp = choose_component(labels, stats, centroids, prev_xy, min_area=min_area)

        if comp is None:
            rows.append((frame_idx, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan))
            prev_xy = None
            continue

        cx, cy = centroids[comp]
        area = stats[comp, cv.CC_STAT_AREA]

        # isolate chosen blob for orientation
        blob = (labels == comp).astype(np.uint8)
        ang = orientation_from_moments(blob)

        # homography to world (mm) if provided
        if homography is not None:
            pt = np.array([cx, cy, 1.0], dtype=np.float64)
            w = homography @ pt
            X = w[0]/w[2]
            Y = w[1]/w[2]
        else:
            X = Y = np.nan

        rows.append((frame_idx, cx, cy, area, ang if ang is not None else np.nan, X, Y, np.nan, np.nan))
        prev_xy = (cx, cy)

    df = pd.DataFrame(rows, columns=[
        "frame", "x_px", "y_px", "area_px2", "angle_deg", "X_mm", "Y_mm", "speed_px_s", "speed_mm_s"
    ])

    # Time & speeds
    df["t_s"] = df["frame"] / float(fps)
    for unit, xcol, ycol in [("px", "x_px", "y_px"), ("mm", "X_mm", "Y_mm")]:
        dx = df[xcol].diff()
        dy = df[ycol].diff()
        dt = df["t_s"].diff()
        speed = np.sqrt(dx*dx + dy*dy) / dt
        df[f"speed_{unit}_s"] = speed

    # Smooth (optional): only on finite entries
    for col in ["x_px", "y_px"]:
        s = df[col].to_numpy()
        mask = np.isfinite(s)
        if mask.sum() >= 9:  # need window size
            s2 = s.copy()
            s2[mask] = savgol_filter(s[mask], window_length= nine(mask.sum()), polyorder=2, mode="interp")
            df[col] = s2

    return df

def nine(n):
    # choose an odd window ~ n/7, clamped
    w = max(5, (n // 7) | 1)
    return min(w, (n if n % 2 == 1 else n-1))

# Example usage:
# df = track_from_masks(r"c:\data\experiment1\masks\frame_*.png", fps=120.0, homography=None)
# df.to_csv(r"c:\data\experiment1\mouse_trajectory.csv", index=False)

def masks_from_video(video_path, out_dir, history=500, varThreshold=16):
    os.makedirs(out_dir, exist_ok=True)
    cap = cv.VideoCapture(video_path)
    fgbg = cv.createBackgroundSubtractorMOG2(history=history, varThreshold=varThreshold, detectShadows=False)
    i = 0
    while True:
        ok, frame = cap.read()
        if not ok: break
        fg = fgbg.apply(frame)
        fg = cv.medianBlur(fg, 5)
        fg = cv.threshold(fg, 0, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)[1]
        cv.imwrite(os.path.join(out_dir, f"mask_{i:06d}.png"), fg)
        i += 1
    cap.release()
    return os.path.join(out_dir, "mask_*.png")


In [ ]:
mask_glob = masks_from_video(r"C:\data\vid.mp4", r"C:\data\masks")
df = track_from_masks(mask_glob, fps=120.0)
# df.to_csv(r"C:\data\mouse_trajectory.csv", index=False)

In [ ]:
import plot_mouse_trajectories_with_maze as plt_w_maze

